In [2]:
import nrrd, cv2
import numpy as np

from utils.common_config import CommonConfig
from utils.tools import Tools

data_path = "D:/workspace/ml-workspace/registration/datasets"
cement_id = 4
custom_ct_image_path = f"{data_path}/sample{cement_id}/ct/matched"

cement_id_range = CommonConfig.get_range(cement_id)

# 读取文件并转化成nrrd文件，以便于3d slicer来进行显示


# 初始切片索引
start_slice_id = cement_id_range[0]
# 终止切片索引
end_slice_id = 1364#cement_id_range[-1]
total_image_num = end_slice_id - start_slice_id

# 下采样间隔
sample_interval = 4
loop_times = total_image_num // sample_interval

# 找到对应的文件前缀
slice_preffix = "slice"

# 分为自定义的文件图像和读取硬盘中的CT图像，两种读取类型是不同的，这是由于CT图像的前缀名和格式不同
custom_mode = False

ct_imgs = []
for i in range(loop_times):
    slice_index = start_slice_id + i * sample_interval

    ori_ct_img = None
    if custom_mode :
        file_name = f"{custom_ct_image_path}/{slice_preffix}_{slice_index}.bmp"
        ori_ct_img = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
    else:
        ori_ct_img = CommonConfig.get_cement_ct_slice(cement_id, slice_index)
    # 对图像进行下采样
    downsp_img = Tools.downsample_image(ori_ct_img, sample_interval)
    ct_imgs.append(downsp_img)

# 叠加构成三维体素
ct3d = np.stack(ct_imgs, axis=0)

nrrd_save_path = f"{data_path}/sample{cement_id}/nrrd"
nrrd_file_name = f"test{cement_id}.nrrd"

# 进行读取并转换成nrrd格式的文件
nrrd.write(f"{nrrd_save_path}/{nrrd_file_name}", ct3d)
